<div style="padding:16px 18px;border:1px solid rgba(0,0,0,.10);border-radius:16px;background:#ffffff;">
  <div style="font-size:28px;font-weight:800;line-height:1.15;">
    Approaching_DS · <span style="color:#009BC8;">Basis: From Pandas to Polars</span>
  </div>
  <div style="margin-top:6px;color:rgba(0,0,0,.65);">
    Subtítulo: Se muestra una revisión de comandos de Pandas y su extensión a Polars, para entornos Big Data
  </div>

  <div style="height:4px;background:#009BC8;border-radius:999px;margin:14px 0 12px 0;"></div>

  <table style="width:100%;border-collapse:collapse;">
    <tr>
      <td style="padding:8px 10px;border:1px solid rgba(0,0,0,.08);border-radius:12px;">
        <b>Autor</b><br><span style="color:rgba(0,0,0,.65);">@romarc98</span>
      </td>
      <td style="padding:8px 10px;border:1px solid rgba(0,0,0,.08);border-radius:12px;">
        <b>Fecha</b><br><span style="color:rgba(0,0,0,.65);">2026-Q1</span>
      </td>
      <td style="padding:8px 10px;border:1px solid rgba(0,0,0,.08);border-radius:12px;">
        <b>Status</b><br><span style="color:rgba(0,0,0,.65);">Draft</span>
      </td>
    </tr>
  </table>
  </div>
</div>


In [3]:
import sys

# Actualizo pip en el entorno del kernel
!{sys.executable} -m pip install -U pip

# Instalo paquetes:
!{sys.executable} -m pip install numpy pandas polars pyarrow matplotlib

   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   -------------------------------- ------- 10.0/12.4 MB 55.7 MB/s eta 0:00:01
   ---------------------------------------- 12.4/12.4 MB 53.4 MB/s  0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 11.1/11.1 MB 60.4 MB/s  0:00:00
   ---------------------------------------- 0.0/802.4 kB ? eta -:--:--
   ---------------------------------------- 802.4/802.4 kB 48.1 MB/s  0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ------------- -------------------------- 14.7/44.5 MB 68.9 MB/s eta 0:00:01
   -------------------- ------------------- 23.1/44.5 MB 68.6 MB/s eta 0:00:01
   ------------------------- -------------- 28.8/44.5 MB 47.0 MB/s eta 0:00:01
   ---------------------------------- ----- 38.0/44.5 MB 45.4 MB/s eta 0:00:01
   ---------------------------------------- 44.5/44.5 MB 43.7 MB/s  0:00:01
   ---------------

In [4]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)
pd.set_option("display.max_rows", 80)

In [5]:
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

def show(df, n=5, title=None):
    if title:
        print(f"\n=== {title} ===")
    display(df.head(n))

def profile_df(df, name="df"):
    print(f"\n--- Profile: {name} ---")
    print("shape:", df.shape)
    print("dtypes:\n", df.dtypes)
    print("missing (%):\n", (df.isna().mean().sort_values(ascending=False) * 100).round(2).head(15))
    print("memory (MB):", (df.memory_usage(deep=True).sum() / (1024**2)).round(3))

In [37]:
import polars as pl

# Config de display en notebook
pl.Config.set_tbl_cols(30)
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_width_chars(140)

def pl_show(df: pl.DataFrame, n=5, title=None):
    if title:
        print(f"\n=== {title} ===")
    display(df.head(n))


def pl_profile(df: pl.DataFrame, name="df"):
    print(f"\n--- Profile: {name} ---")
    print("shape:", df.shape)
    print("dtypes:\n", df.schema)
    nulls = df.null_count().transpose(include_header=True).rename({"column_0": "col", "column_1": "nulls"})
    nulls = nulls.with_columns((pl.col("nulls") / df.height * 100).round(2).alias("null_%")).sort("null_%", descending=True)
    print("missing top:\n", nulls.head(15))
    print("estimated size (MB):", round(df.estimated_size() / (1024**2), 3))


In [6]:
def make_synthetic_data(
    n_customers=50_000,
    n_products=2_000,
    n_txn=400_000,
    start="2023-01-01",
    end="2025-12-31",
    seed=42,
):
    rng = np.random.default_rng(seed)

    # Customers (dimension)
    customer_id = np.arange(1, n_customers + 1)

    segments = np.array(["Mass", "Affluent", "SME", "Corp"])
    cities = np.array(["Barcelona", "Madrid", "Valencia", "Sevilla", "Bilbao", "Zaragoza", "Malaga"])
    channels = np.array(["Branch", "App", "Web", "CallCenter"])
    countries = np.array(["ES", "PT", "FR", "IT", "DE"])

    signup_dates = pd.to_datetime(start) + pd.to_timedelta(
        rng.integers(0, (pd.Timestamp(end) - pd.Timestamp(start)).days + 1, size=n_customers),
        unit="D"
    )

    df_customers = pd.DataFrame({
        "customer_id": customer_id,
        "segment": rng.choice(segments, size=n_customers, p=[0.62, 0.20, 0.15, 0.03]),
        "city": rng.choice(cities, size=n_customers),
        "country": rng.choice(countries, size=n_customers, p=[0.86, 0.03, 0.03, 0.04, 0.04]),
        "age": rng.integers(18, 85, size=n_customers),
        "income_monthly": np.round(rng.lognormal(mean=7.6, sigma=0.45, size=n_customers), 2),  # aprox
        "signup_date": signup_dates,
        "is_active": rng.choice([True, False], size=n_customers, p=[0.92, 0.08]),
    })

    # Introducimos missing / ruido:
    miss_mask = rng.random(n_customers) < 0.03
    df_customers.loc[miss_mask, "income_monthly"] = np.nan

    # Texto (para str methods / regex):
    notes_vocab = np.array(["promo", "vip", "late_payer", "student", "mortgage", "refund", "travel", "complaint"])
    df_customers["notes"] = rng.choice(notes_vocab, size=n_customers)
    df_customers.loc[rng.random(n_customers) < 0.04, "notes"] = None

    # Score continuo con correlación leve con segmento/edad:
    seg_bias = df_customers["segment"].map({"Mass": 0.1, "Affluent": 0.25, "SME": 0.18, "Corp": 0.3}).astype(float)
    df_customers["risk_score"] = np.clip(
        rng.normal(loc=0.45, scale=0.18, size=n_customers) - 0.002*(df_customers["age"]-45) + seg_bias,
        0, 1
    ).round(4)

    # Products (dimension)
    product_id = np.arange(1, n_products + 1)
    categories = np.array(["Cards", "Loans", "Investments", "Insurance", "Payments", "Subscriptions"])
    is_digital = rng.choice([True, False], size=n_products, p=[0.55, 0.45])

    base_price = rng.lognormal(mean=2.8, sigma=0.7, size=n_products)
    df_products = pd.DataFrame({
        "product_id": product_id,
        "category": rng.choice(categories, size=n_products),
        "is_digital": is_digital,
        "base_fee": np.round(base_price, 2),
    })

    # Transactions (fact)
    txn_id = np.arange(1, n_txn + 1)

    # timestamps uniformes entre start y end
    start_ts = pd.Timestamp(start)
    end_ts = pd.Timestamp(end)
    total_seconds = int((end_ts - start_ts).total_seconds())
    txn_ts = start_ts + pd.to_timedelta(rng.integers(0, total_seconds, size=n_txn), unit="s")

    statuses = np.array(["SETTLED", "DECLINED", "REVERSED", "PENDING"])
    currencies = np.array(["EUR", "USD", "GBP"])

    df_txn = pd.DataFrame({
        "txn_id": txn_id,
        "customer_id": rng.choice(customer_id, size=n_txn),
        "product_id": rng.choice(product_id, size=n_txn),
        "channel": rng.choice(channels, size=n_txn, p=[0.25, 0.45, 0.20, 0.10]),
        "status": rng.choice(statuses, size=n_txn, p=[0.86, 0.06, 0.03, 0.05]),
        "currency": rng.choice(currencies, size=n_txn, p=[0.92, 0.05, 0.03]),
        "quantity": rng.integers(1, 6, size=n_txn),
        "txn_ts": txn_ts,
    })

    # amount con heavy tail + ajustes por categoría (lo haremos tras merge)
    raw_amount = rng.lognormal(mean=3.4, sigma=1.0, size=n_txn)
    df_txn["amount"] = np.round(raw_amount, 2)

    # discount / flags
    df_txn["discount_pct"] = np.where(rng.random(n_txn) < 0.25, np.round(rng.uniform(0.01, 0.35, size=n_txn), 3), 0.0)
    df_txn["is_refund"] = rng.random(n_txn) < 0.03

    # Missing artificial
    df_txn.loc[rng.random(n_txn) < 0.01, "channel"] = None

    # Ajuste de amount si refund
    df_txn.loc[df_txn["is_refund"], "amount"] *= -1

    # fx_rate simple
    fx = {"EUR": 1.0, "USD": 0.92, "GBP": 1.15}
    df_txn["fx_rate_to_eur"] = df_txn["currency"].map(fx).astype(float)
    df_txn["amount_eur"] = np.round(df_txn["amount"] * df_txn["fx_rate_to_eur"], 2)

    # Monthly risk time series per customer
    months = pd.date_range(pd.Timestamp(start).to_period("M").start_time,
                          pd.Timestamp(end).to_period("M").start_time,
                          freq="MS")
    # Para no explotar memoria: sampleamos subset de clientes para la TS (puedes poner todos si quieres)
    ts_customers = rng.choice(customer_id, size=min(n_customers, 40_000), replace=False)

    idx = pd.MultiIndex.from_product([ts_customers, months], names=["customer_id", "month"])
    df_risk_monthly = pd.DataFrame(index=idx).reset_index()

    # EAD/PD/LGD sintéticos
    base_ead = rng.lognormal(mean=8.2, sigma=0.8, size=len(ts_customers))  # por cliente
    base_pd = np.clip(rng.normal(0.03, 0.02, size=len(ts_customers)), 0.001, 0.25)

    cust_map = pd.DataFrame({"customer_id": ts_customers, "base_ead": base_ead, "base_pd": base_pd})
    df_risk_monthly = df_risk_monthly.merge(cust_map, on="customer_id", how="left")

    t = (df_risk_monthly["month"] - df_risk_monthly["month"].min()).dt.days / 30.0
    season = 0.08 * np.sin(2*np.pi*t/12)

    df_risk_monthly["ead"] = np.round(df_risk_monthly["base_ead"] * (1 + 0.10*season + rng.normal(0, 0.05, size=len(df_risk_monthly))), 2)
    df_risk_monthly["pd"] = np.clip(df_risk_monthly["base_pd"] * (1 + 0.35*season + rng.normal(0, 0.15, size=len(df_risk_monthly))), 0.0005, 0.35).round(5)
    df_risk_monthly["lgd"] = np.clip(rng.normal(0.45, 0.12, size=len(df_risk_monthly)), 0.05, 0.95).round(4)

    # stage como función burda de pd:
    df_risk_monthly["stage"] = pd.cut(
        df_risk_monthly["pd"],
        bins=[-np.inf, 0.02, 0.08, np.inf],
        labels=["Stage1", "Stage2", "Stage3"]
    ).astype("string")

    # Limpieza columnas auxiliares
    df_risk_monthly = df_risk_monthly.drop(columns=["base_ead", "base_pd"])

    return df_customers, df_products, df_txn, df_risk_monthly

In [7]:

df_customers, df_products, df_txn, df_risk_monthly = make_synthetic_data(
    n_customers=80_000,
    n_products=3_000,
    n_txn=600_000,
    start="2023-01-01",
    end="2025-12-31",
    seed=RANDOM_SEED
)

profile_df(df_customers, "df_customers")
profile_df(df_products, "df_products")
profile_df(df_txn, "df_txn")
profile_df(df_risk_monthly, "df_risk_monthly")

show(df_customers, title="Customers")
show(df_products, title="Products")
show(df_txn, title="Transactions")
show(df_risk_monthly, title="Risk monthly")



--- Profile: df_customers ---
shape: (80000, 10)
dtypes:
 customer_id                int64
segment                   object
city                      object
country                   object
age                        int64
income_monthly           float64
signup_date       datetime64[ns]
is_active                   bool
notes                     object
risk_score               float64
dtype: object
missing (%):
 notes             3.94
income_monthly    2.99
segment           0.00
customer_id       0.00
country           0.00
city              0.00
age               0.00
signup_date       0.00
is_active         0.00
risk_score        0.00
dtype: float64
memory (MB): 19.554

--- Profile: df_products ---
shape: (3000, 4)
dtypes:
 product_id      int64
category       object
is_digital       bool
base_fee      float64
dtype: object
missing (%):
 product_id    0.0
category      0.0
is_digital    0.0
base_fee      0.0
dtype: float64
memory (MB): 0.213

--- Profile: df_txn ---
shape: (600000,

,customer_id,segment,city,country,age,income_monthly,signup_date,is_active,notes,risk_score
0,1,Mass,Sevilla,ES,78,2007.23,2023-04-08,True,vip,0.0712
1,2,Mass,Bilbao,ES,70,2001.93,2025-04-28,True,mortgage,0.6283
2,3,Mass,Sevilla,ES,57,3268.38,2024-12-18,True,vip,0.6825
3,4,Mass,Zaragoza,ES,79,1533.57,2024-04-26,True,mortgage,0.8930
4,5,Corp,Valencia,ES,79,585.07,2024-04-19,True,vip,0.7127



=== Products ===


,product_id,category,is_digital,base_fee
0,1,Payments,True,31.01
1,2,Loans,True,19.72
2,3,Investments,True,39.73
3,4,Insurance,True,6.20
4,5,Investments,False,19.06



=== Transactions ===


,txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts,amount,discount_pct,is_refund,fx_rate_to_eur,amount_eur
0,1,11700,1401,Branch,SETTLED,USD,5,2023-01-13 09:03:32,95.71,0.046,False,0.92,88.05
1,2,1086,1700,Web,SETTLED,EUR,2,2023-05-12 13:45:22,22.60,0.000,False,1.00,22.60
2,3,68944,2594,Web,SETTLED,EUR,1,2024-02-06 14:22:15,11.60,0.000,False,1.00,11.60
3,4,27155,2179,App,DECLINED,EUR,1,2024-08-03 04:03:22,60.91,0.000,False,1.00,60.91
4,5,70629,2643,Branch,SETTLED,EUR,1,2023-08-30 18:44:27,34.25,0.000,False,1.00,34.25



=== Risk monthly ===


,customer_id,month,ead,pd,lgd,stage
0,33331,2023-01-01,2754.20,0.03246,0.5822,Stage2
1,33331,2023-02-01,2751.60,0.02869,0.2160,Stage2
2,33331,2023-03-01,2840.74,0.02748,0.5691,Stage2
3,33331,2023-04-01,2583.19,0.03673,0.4261,Stage2
4,33331,2023-05-01,2846.99,0.03516,0.5054,Stage2


In [41]:
# Conversión directa (rápida para prototipar; para benchmarks “serios”, mejor leer Parquet)
costumers_pl  = pl.from_pandas(df_customers)
products_pl  = pl.from_pandas(df_products)
risk_pl      = pl.from_pandas(df_risk_monthly)
txn_pl       = pl.from_pandas(df_txn)

pl_show(products_pl,  title="Products (Polars)")
pl_show(risk_pl,      title="Risk monthly (Polars)")
pl_show(txn_pl,  title="Transactions (Polars)")
pl_show(costumers_pl,      title="Costmers(Polars)")


=== Products (Polars) ===


product_id,category,is_digital,base_fee
i64,str,bool,f64
1,"""Payments""",true,31.01
2,"""Loans""",true,19.72
3,"""Investments""",true,39.73
4,"""Insurance""",true,6.2
5,"""Investments""",false,19.06



=== Risk monthly (Polars) ===


customer_id,month,ead,pd,lgd,stage
i64,datetime[ns],f64,f64,f64,str
33331,2023-01-01 00:00:00,2754.2,0.03246,0.5822,"""Stage2"""
33331,2023-02-01 00:00:00,2751.6,0.02869,0.216,"""Stage2"""
33331,2023-03-01 00:00:00,2840.74,0.02748,0.5691,"""Stage2"""
33331,2023-04-01 00:00:00,2583.19,0.03673,0.4261,"""Stage2"""
33331,2023-05-01 00:00:00,2846.99,0.03516,0.5054,"""Stage2"""



=== Transactions (Polars) ===


txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts,amount,discount_pct,is_refund,fx_rate_to_eur,amount_eur,txn_date,txn_month
i64,i64,i64,str,str,str,i64,datetime[ns],f64,f64,bool,f64,f64,date,str
1,11700,1401,"""Branch""","""SETTLED""","""USD""",5,2023-01-13 09:03:32,95.71,0.046,false,0.92,88.05,2023-01-13,"""2023-01"""
2,1086,1700,"""Web""","""SETTLED""","""EUR""",2,2023-05-12 13:45:22,22.6,0.0,false,1.0,22.6,2023-05-12,"""2023-05"""
3,68944,2594,"""Web""","""SETTLED""","""EUR""",1,2024-02-06 14:22:15,11.6,0.0,false,1.0,11.6,2024-02-06,"""2024-02"""
4,27155,2179,"""App""","""DECLINED""","""EUR""",1,2024-08-03 04:03:22,60.91,0.0,false,1.0,60.91,2024-08-03,"""2024-08"""
5,70629,2643,"""Branch""","""SETTLED""","""EUR""",1,2023-08-30 18:44:27,34.25,0.0,false,1.0,34.25,2023-08-30,"""2023-08"""



=== Costmers(Polars) ===


customer_id,segment,city,country,age,income_monthly,signup_date,is_active,notes,risk_score,notes_clean
i64,str,str,str,i64,f64,datetime[ns],bool,str,f64,str
1,"""Mass""","""Sevilla""","""ES""",78,2007.23,2023-04-08 00:00:00,true,"""vip""",0.0712,"""vip"""
2,"""Mass""","""Bilbao""","""ES""",70,2001.93,2025-04-28 00:00:00,true,"""mortgage""",0.6283,"""mortgage"""
3,"""Mass""","""Sevilla""","""ES""",57,3268.38,2024-12-18 00:00:00,true,"""vip""",0.6825,"""vip"""
4,"""Mass""","""Zaragoza""","""ES""",79,1533.57,2024-04-26 00:00:00,true,"""mortgage""",0.893,"""mortgage"""
5,"""Corp""","""Valencia""","""ES""",79,585.07,2024-04-19 00:00:00,true,"""vip""",0.7127,"""vip"""


In [8]:
df_txn.describe(include="all").T.head(25)

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
txn_id,600000.0,NaN,NaN,NaN,300000.5,1.0,150000.75,300000.5,450000.25,600000.0,173205.225094
customer_id,600000.0,NaN,NaN,NaN,40027.555807,1.0,20057.0,40033.0,60022.0,80000.0,23075.570741
product_id,600000.0,NaN,NaN,NaN,1500.691327,1.0,750.0,1501.0,2250.0,3000.0,865.881864
channel,593845,4,App,267481,NaN,NaN,NaN,NaN,NaN,NaN,NaN
status,600000,4,SETTLED,515917,NaN,NaN,NaN,NaN,NaN,NaN,NaN
currency,600000,3,EUR,552144,NaN,NaN,NaN,NaN,NaN,NaN,NaN
quantity,600000.0,NaN,NaN,NaN,2.997965,1.0,2.0,3.0,4.0,5.0,1.413881
txn_ts,600000,NaN,NaN,NaN,2024-07-01 21:21:25.004173056,2023-01-01 00:00:11,2023-10-02 04:52:04.750000128,2024-07-01 21:38:30.500000,2025-04-01 20:53:29.750000128,2025-12-30 23:59:33,NaN
amount,600000.0,NaN,NaN,NaN,46.410937,-2233.47,14.18,28.84,57.43,3532.85,66.617758
discount_pct,600000.0,NaN,NaN,NaN,0.044806,0.0,0.0,0.0,0.0,0.35,0.091901


In [42]:
txn_pl.describe()

statistic,txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts,amount,discount_pct,is_refund,fx_rate_to_eur,amount_eur,txn_date,txn_month
str,f64,f64,f64,str,str,str,f64,str,f64,f64,f64,f64,f64,str,str
"""count""",600000.0,600000.0,600000.0,"""593845""","""600000""","""600000""",600000.0,"""600000""",600000.0,600000.0,600000.0,600000.0,600000.0,"""600000""","""600000"""
"""null_count""",0.0,0.0,0.0,"""6155""","""0""","""0""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0"""
"""mean""",300000.5,40027.555807,1500.691327,null,null,null,2.997965,"""2024-07-01 21:21:25.004173""",46.410937,0.044806,0.02965,1.000558,46.436601,"""2024-07-01 09:21:25.632000""",null
"""std""",173205.225094,23075.570741,865.881864,null,null,null,1.413881,null,66.617758,0.091901,null,0.031557,66.686325,null,null
"""min""",1.0,1.0,1.0,"""App""","""DECLINED""","""EUR""",1.0,"""2023-01-01 00:00:11""",-2233.47,0.0,0.0,0.92,-2233.47,"""2023-01-01""","""2023-01"""
"""25%""",150001.0,20057.0,750.0,null,null,null,2.0,"""2023-10-02 04:53:18""",14.18,0.0,null,1.0,14.17,"""2023-10-02""",null
"""50%""",300001.0,40033.0,1501.0,null,null,null,3.0,"""2024-07-01 21:38:36""",28.84,0.0,null,1.0,28.85,"""2024-07-01""",null
"""75%""",450000.0,60022.0,2250.0,null,null,null,4.0,"""2025-04-01 20:53:24""",57.43,0.0,null,1.0,57.45,"""2025-04-01""",null
"""max""",600000.0,80000.0,3000.0,"""Web""","""SETTLED""","""USD""",5.0,"""2025-12-30 23:59:33""",3532.85,0.35,1.0,1.15,3532.85,"""2025-12-30""","""2025-12"""


In [9]:
# cardinalidades y duplicados
summary = pd.DataFrame({
    "nunique": df_txn.nunique(dropna=True),
    "nulls": df_txn.isna().sum(),
    "null_%": (df_txn.isna().mean() * 100).round(2)
}).sort_values("null_%", ascending=False)

summary

,nunique,nulls,null_%
channel,4,6155,1.03
txn_id,600000,0,0.00
customer_id,79957,0,0.00
product_id,3000,0,0.00
status,4,0,0.00
currency,3,0,0.00
quantity,5,0,0.00
txn_ts,598091,0,0.00
amount,39970,0,0.00
discount_pct,342,0,0.00


In [10]:
df_txn.duplicated(subset=["txn_id"]).sum()

np.int64(0)

In [43]:
nulls = txn_pl.null_count()
nulls

txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts,amount,discount_pct,is_refund,fx_rate_to_eur,amount_eur,txn_date,txn_month
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,6155,0,0,0,0,0,0,0,0,0,0,0


In [44]:
# nunique por columna (ojo: puede ser costoso en tablas grandes)
nunique = txn_pl.select([pl.col(c).n_unique().alias(c) for c in txn_pl.columns])
nunique


txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts,amount,discount_pct,is_refund,fx_rate_to_eur,amount_eur,txn_date,txn_month
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
600000,79957,3000,5,4,3,5,598091,39970,342,2,3,40018,1095,36


In [11]:
# loc / iloc
sample_rows = df_txn.iloc[:10, :8]
sample_rows

,txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts
0,1,11700,1401,Branch,SETTLED,USD,5,2023-01-13 09:03:32
1,2,1086,1700,Web,SETTLED,EUR,2,2023-05-12 13:45:22
2,3,68944,2594,Web,SETTLED,EUR,1,2024-02-06 14:22:15
3,4,27155,2179,App,DECLINED,EUR,1,2024-08-03 04:03:22
4,5,70629,2643,Branch,SETTLED,EUR,1,2023-08-30 18:44:27
5,6,24230,1559,Web,SETTLED,EUR,1,2025-06-30 15:39:57
6,7,78928,2911,App,SETTLED,EUR,3,2023-03-14 11:49:09
7,8,42063,885,Branch,SETTLED,EUR,3,2023-01-24 15:15:30
8,9,33475,1134,CallCenter,SETTLED,EUR,1,2024-01-03 16:10:41
9,10,2028,2290,Branch,SETTLED,EUR,5,2025-07-15 16:29:29


In [45]:
# "iloc": head / slice
txn_pl.select(txn_pl.columns[:8]).head(10)


txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts
i64,i64,i64,str,str,str,i64,datetime[ns]
1,11700,1401,"""Branch""","""SETTLED""","""USD""",5,2023-01-13 09:03:32
2,1086,1700,"""Web""","""SETTLED""","""EUR""",2,2023-05-12 13:45:22
3,68944,2594,"""Web""","""SETTLED""","""EUR""",1,2024-02-06 14:22:15
4,27155,2179,"""App""","""DECLINED""","""EUR""",1,2024-08-03 04:03:22
5,70629,2643,"""Branch""","""SETTLED""","""EUR""",1,2023-08-30 18:44:27
6,24230,1559,"""Web""","""SETTLED""","""EUR""",1,2025-06-30 15:39:57
7,78928,2911,"""App""","""SETTLED""","""EUR""",3,2023-03-14 11:49:09
8,42063,885,"""Branch""","""SETTLED""","""EUR""",3,2023-01-24 15:15:30
9,33475,1134,"""CallCenter""","""SETTLED""","""EUR""",1,2024-01-03 16:10:41


In [12]:
# boolean masks
mask = (df_txn["status"].eq("SETTLED")) & (df_txn["amount_eur"].abs() > 500)
df_big = df_txn.loc[mask, ["txn_id", "customer_id", "amount_eur", "currency", "txn_ts", "status"]]
df_big.head()

,txn_id,customer_id,amount_eur,currency,txn_ts,status
90,91,34873,510.70,EUR,2024-03-15 08:35:14,SETTLED
168,169,57026,728.28,EUR,2023-07-08 10:04:00,SETTLED
399,400,9280,635.90,EUR,2025-08-17 16:07:36,SETTLED
757,758,59496,967.04,EUR,2024-09-18 06:42:31,SETTLED
886,887,45148,1155.56,EUR,2024-01-23 16:01:44,SETTLED


In [ ]:
# query (útil para expresiones "tipo SQL")
df_txn.query("status == 'SETTLED' and amount_eur > 1000")[["txn_id", "amount_eur", "currency", "status"]].head()

,txn_id,amount_eur,currency,status
886,887,1155.56,EUR,SETTLED
5073,5074,1318.27,USD,SETTLED
5338,5339,1127.05,EUR,SETTLED
6583,6584,1340.37,EUR,SETTLED
13649,13650,1153.90,EUR,SETTLED


In [47]:
# boolean mask equivalente
df_big_pl = (
    txn_pl
    .filter((pl.col("status") == "SETTLED") & (pl.col("amount_eur").abs() > 500))
    .select(["txn_id", "customer_id", "amount_eur", "currency", "txn_ts", "status"])
)
df_big_pl.head()

txn_id,customer_id,amount_eur,currency,txn_ts,status
i64,i64,f64,str,datetime[ns],str
91,34873,510.7,"""EUR""",2024-03-15 08:35:14,"""SETTLED"""
169,57026,728.28,"""EUR""",2023-07-08 10:04:00,"""SETTLED"""
400,9280,635.9,"""EUR""",2025-08-17 16:07:36,"""SETTLED"""
758,59496,967.04,"""EUR""",2024-09-18 06:42:31,"""SETTLED"""
887,45148,1155.56,"""EUR""",2024-01-23 16:01:44,"""SETTLED"""


In [14]:
# at/iat (acceso escalar rápido)
i = 0
df_txn.at[i, "status"], df_txn.iat[i, df_txn.columns.get_loc("amount_eur")]

('SETTLED', np.float64(88.05))

In [48]:
txn_pl.schema

Schema([('txn_id', Int64),
        ('customer_id', Int64),
        ('product_id', Int64),
        ('channel', String),
        ('status', String),
        ('currency', String),
        ('quantity', Int64),
        ('txn_ts', Datetime(time_unit='ns', time_zone=None)),
        ('amount', Float64),
        ('discount_pct', Float64),
        ('is_refund', Boolean),
        ('fx_rate_to_eur', Float64),
        ('amount_eur', Float64),
        ('txn_date', Date),
        ('txn_month', String)])

In [ ]:
df_customers.dtypes

customer_id                int64
segment                   object
city                      object
country                   object
age                        int64
income_monthly           float64
signup_date       datetime64[ns]
is_active                   bool
notes                     object
risk_score               float64
dtype: object

In [16]:
# convert_dtypes (mejora dtypes "nullable")
df_customers2 = df_customers.convert_dtypes()
df_customers2.dtypes

customer_id                Int64
segment           string[python]
city              string[python]
country           string[python]
age                        Int64
income_monthly           Float64
signup_date       datetime64[ns]
is_active                boolean
notes             string[python]
risk_score               Float64
dtype: object

In [17]:
# category para columnas de baja cardinalidad (mem/perf)
df_customers_cat = df_customers.copy()
for col in ["segment", "city", "country", "notes"]:
    df_customers_cat[col] = df_customers_cat[col].astype("category")

profile_df(df_customers, "df_customers (orig)")
profile_df(df_customers_cat, "df_customers (with category)")



--- Profile: df_customers (orig) ---
shape: (80000, 10)
dtypes:
 customer_id                int64
segment                   object
city                      object
country                   object
age                        int64
income_monthly           float64
signup_date       datetime64[ns]
is_active                   bool
notes                     object
risk_score               float64
dtype: object
missing (%):
 notes             3.94
income_monthly    2.99
segment           0.00
customer_id       0.00
country           0.00
city              0.00
age               0.00
signup_date       0.00
is_active         0.00
risk_score        0.00
dtype: float64
memory (MB): 19.554

--- Profile: df_customers (with category) ---
shape: (80000, 10)
dtypes:
 customer_id                int64
segment                 category
city                    category
country                 category
age                        int64
income_monthly           float64
signup_date       datetime64[ns]
is_ac

In [18]:
# datetime features
df_txn["txn_date"] = df_txn["txn_ts"].dt.date
df_txn["txn_month"] = df_txn["txn_ts"].dt.to_period("M").astype(str)
df_txn[["txn_ts", "txn_date", "txn_month"]].head()


,txn_ts,txn_date,txn_month
0,2023-01-13 09:03:32,2023-01-13,2023-01
1,2023-05-12 13:45:22,2023-05-12,2023-05
2,2024-02-06 14:22:15,2024-02-06,2024-02
3,2024-08-03 04:03:22,2024-08-03,2024-08
4,2023-08-30 18:44:27,2023-08-30,2023-08


In [53]:
txn_pl_dt = txn_pl.with_columns([
    pl.col("txn_ts").cast(pl.Datetime).alias("txn_ts"),
    pl.col("txn_ts").cast(pl.Datetime).dt.date().alias("txn_date"),
    pl.col("txn_ts").cast(pl.Datetime).dt.truncate("1mo").alias("txn_month_start"),
])
txn_pl_dt.select(["txn_ts", "txn_date", "txn_month_start"]).head()


txn_ts,txn_date,txn_month_start
datetime[μs],date,datetime[μs]
2023-01-13 09:03:32,2023-01-13,2023-01-01 00:00:00
2023-05-12 13:45:22,2023-05-12,2023-05-01 00:00:00
2024-02-06 14:22:15,2024-02-06,2024-02-01 00:00:00
2024-08-03 04:03:22,2024-08-03,2024-08-01 00:00:00
2023-08-30 18:44:27,2023-08-30,2023-08-01 00:00:00


In [19]:
# string methods + regex
df_customers["notes_clean"] = df_customers["notes"].fillna("").str.lower().str.replace(r"[^a-z_]", "", regex=True)
df_customers[["notes", "notes_clean"]].head(10)


,notes,notes_clean
0,vip,vip
1,mortgage,mortgage
2,vip,vip
3,mortgage,mortgage
4,vip,vip
5,student,student
6,mortgage,mortgage
7,promo,promo
8,refund,refund
9,refund,refund


In [54]:
customers_pl_txt = costumers_pl.with_columns([
    pl.col("notes")
      .fill_null("")
      .str.to_lowercase()
      .str.replace_all(r"[^a-z_]", "")
      .alias("notes_clean")
])

customers_pl_txt.select(["notes", "notes_clean"]).head(10)


notes,notes_clean
str,str
"""vip""","""vip"""
"""mortgage""","""mortgage"""
"""vip""","""vip"""
"""mortgage""","""mortgage"""
"""vip""","""vip"""
"""student""","""student"""
"""mortgage""","""mortgage"""
"""promo""","""promo"""
"""refund""","""refund"""


In [20]:
# dónde faltan datos
df_txn.isna().mean().sort_values(ascending=False).head(10)


channel         0.010258
customer_id     0.000000
txn_id          0.000000
product_id      0.000000
status          0.000000
currency        0.000000
quantity        0.000000
txn_ts          0.000000
amount          0.000000
discount_pct    0.000000
dtype: float64

In [21]:
# fillna simple
df_txn_filled = df_txn.copy()
df_txn_filled["channel"] = df_txn_filled["channel"].fillna("Unknown")

# imputación condicional: income por segmento con mediana
df_customers_imp = df_customers.copy()
med_income_by_seg = df_customers_imp.groupby("segment")["income_monthly"].median()
df_customers_imp["income_monthly"] = df_customers_imp["income_monthly"].fillna(
    df_customers_imp["segment"].map(med_income_by_seg)
)

df_customers_imp[["segment", "income_monthly"]].head()


,segment,income_monthly
0,Mass,2007.23
1,Mass,2001.93
2,Mass,3268.38
3,Mass,1533.57
4,Corp,585.07


In [56]:
txn_pl_filled = txn_pl.with_columns([
    pl.col("channel").fill_null("Unknown").alias("channel")
])

med_income = customers_pl.group_by("segment").agg(
    pl.col("income_monthly").median().alias("med_income")
)

customers_pl_imp = (
    customers_pl
    .join(med_income, on="segment", how="left")
    .with_columns([
        pl.when(pl.col("income_monthly").is_null())
          .then(pl.col("med_income"))
          .otherwise(pl.col("income_monthly"))
          .alias("income_monthly")
    ])
    .drop("med_income")
)

customers_pl_imp.select(["segment", "income_monthly"]).head(10)


segment,income_monthly
str,f64
"""Mass""",2007.23
"""Mass""",2001.93
"""Mass""",3268.38
"""Mass""",1533.57
"""Corp""",585.07
"""Affluent""",2158.24
"""Mass""",979.05
"""Mass""",1815.9
"""Mass""",816.55


In [22]:
# where / mask
df_txn_clip = df_txn.copy()
df_txn_clip["amount_eur_clipped"] = df_txn_clip["amount_eur"].clip(lower=-5_000, upper=5_000)
df_txn_clip[["amount_eur", "amount_eur_clipped"]].head()


,amount_eur,amount_eur_clipped
0,88.05,88.05
1,22.60,22.60
2,11.60,11.60
3,60.91,60.91
4,34.25,34.25


In [57]:
txn_pl_clip = txn_pl.with_columns([
    pl.col("amount_eur").clip(-5000, 5000).alias("amount_eur_clipped")
])
txn_pl_clip.select(["amount_eur", "amount_eur_clipped"]).head()


amount_eur,amount_eur_clipped
f64,f64
88.05,88.05
22.6,22.6
11.6,11.6
60.91,60.91
34.25,34.25


In [23]:
df_txn_fe = df_txn.copy()

# importe neto tras descuento (solo settled)
df_txn_fe["net_amount_eur"] = np.where(
    df_txn_fe["status"].eq("SETTLED"),
    df_txn_fe["amount_eur"] * (1 - df_txn_fe["discount_pct"]),
    0.0
).round(2)

# bins / discretización
df_customers_fe = df_customers_imp.copy()
df_customers_fe["age_band"] = pd.cut(
    df_customers_fe["age"], bins=[17, 25, 35, 45, 60, 90],
    labels=["18-25", "26-35", "36-45", "46-60", "60+"]
)

# qcut (cuantiles)
df_customers_fe["risk_decile"] = pd.qcut(df_customers_fe["risk_score"], q=10, labels=False, duplicates="drop")

df_customers_fe[["age", "age_band", "risk_score", "risk_decile"]].head(10)


,age,age_band,risk_score,risk_decile
0,78,60+,0.0712,0
1,70,60+,0.6283,5
2,57,46-60,0.6825,6
3,79,60+,0.8930,9
4,79,60+,0.7127,7
5,41,36-45,0.9839,9
6,71,60+,0.3589,1
7,73,60+,0.4923,3
8,40,36-45,0.7445,7
9,78,60+,0.4538,2


In [24]:
# KPIs por customer_id
kpi = (
    df_txn_fe[df_txn_fe["status"].eq("SETTLED")]
    .groupby("customer_id", as_index=False)
    .agg(
        n_txn=("txn_id", "count"),
        total_net=("net_amount_eur", "sum"),
        avg_net=("net_amount_eur", "mean"),
        last_txn=("txn_ts", "max"),
    )
)

show(kpi, title="KPIs por cliente")



=== KPIs por cliente ===


,customer_id,n_txn,total_net,avg_net,last_txn
0,1,6,187.18,31.196667,2025-02-17 22:23:43
1,2,3,72.79,24.263333,2024-05-16 22:04:52
2,3,6,418.71,69.785000,2025-07-25 14:31:09
3,4,8,334.86,41.857500,2025-11-26 03:42:33
4,5,5,607.21,121.442000,2025-12-17 12:16:24


In [61]:
kpi_pl = (
     txn_pl_fe
    .filter(pl.col("status") == "SETTLED")
    .group_by("customer_id")
    .agg([
        pl.len().alias("n_txn"),
        pl.col("net_amount_eur").sum().alias("total_net"),
        pl.col("net_amount_eur").mean().alias("avg_net"),
        pl.col("txn_ts").max().alias("last_txn"),
    ])
)

kpi_pl.head()


customer_id,n_txn,total_net,avg_net,last_txn
i64,u32,f64,f64,datetime[ns]
28454,8,633.04,79.13,2025-10-13 02:45:48
72369,9,281.19,31.243333,2025-10-15 00:00:19
39982,8,298.96,37.37,2025-11-14 01:43:28
28993,8,215.44,26.93,2025-11-23 05:24:42
15351,8,310.92,38.865,2025-08-26 18:48:20


In [25]:
# transform: añade métricas a nivel fila (sin perder granularidad)
df_txn_k = df_txn_fe.merge(kpi[["customer_id", "total_net", "n_txn"]], on="customer_id", how="left")

# ratio del importe neto sobre el total del cliente
df_txn_k["share_of_customer"] = (df_txn_k["net_amount_eur"] / df_txn_k["total_net"]).replace([np.inf, -np.inf], np.nan)
df_txn_k[["customer_id", "net_amount_eur", "total_net", "share_of_customer"]].head()


,customer_id,net_amount_eur,total_net,share_of_customer
0,11700,84.00,233.15,0.360283
1,1086,22.60,129.77,0.174154
2,68944,11.60,323.85,0.035819
3,27155,0.00,316.35,0.000000
4,70629,34.25,214.17,0.159920


In [26]:
# groupby + rolling (window functions)
df_txn_sorted = df_txn_fe.sort_values(["customer_id", "txn_ts"])
df_txn_sorted["rolling_10_txn_mean"] = (
    df_txn_sorted.groupby("customer_id")["net_amount_eur"]
    .rolling(window=10, min_periods=3)
    .mean()
    .reset_index(level=0, drop=True)
)

df_txn_sorted[["customer_id", "txn_ts", "net_amount_eur", "rolling_10_txn_mean"]].head(20)


,customer_id,txn_ts,net_amount_eur,rolling_10_txn_mean
21000,1,2023-02-17 12:55:48,29.52,NaN
455974,1,2023-04-18 02:30:05,25.01,NaN
527063,1,2023-07-22 17:53:47,0.00,18.176667
578025,1,2024-09-07 05:09:40,19.78,18.577500
197787,1,2024-10-11 06:53:13,11.54,17.170000
6286,1,2024-10-25 23:07:07,20.82,17.778333
146302,1,2025-02-17 22:23:43,80.51,26.740000
172668,2,2023-03-08 22:15:42,0.00,NaN
14288,2,2023-07-16 03:13:28,31.36,NaN
288002,2,2024-02-07 20:44:55,27.74,19.700000


In [27]:
# Enriquecemos transacciones con clientes y productos
df_txn_enriched = (
    df_txn_fe
    .merge(df_customers_fe, on="customer_id", how="left", validate="many_to_one")
    .merge(df_products, on="product_id", how="left", validate="many_to_one")
)

profile_df(df_txn_enriched, "df_txn_enriched")
df_txn_enriched.head()



--- Profile: df_txn_enriched ---
shape: (600000, 31)
dtypes:
 txn_id                     int64
customer_id                int64
product_id                 int64
channel                   object
status                    object
currency                  object
quantity                   int64
txn_ts            datetime64[ns]
amount                   float64
discount_pct             float64
is_refund                   bool
fx_rate_to_eur           float64
amount_eur               float64
txn_date                  object
txn_month                 object
net_amount_eur           float64
segment                   object
city                      object
country                   object
age                        int64
income_monthly           float64
signup_date       datetime64[ns]
is_active                   bool
notes                     object
risk_score               float64
notes_clean               object
age_band                category
risk_decile                int64
category     

,txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts,amount,discount_pct,is_refund,fx_rate_to_eur,amount_eur,txn_date,txn_month,net_amount_eur,segment,city,country,age,income_monthly,signup_date,is_active,notes,risk_score,notes_clean,age_band,risk_decile,category,is_digital,base_fee
0,1,11700,1401,Branch,SETTLED,USD,5,2023-01-13 09:03:32,95.71,0.046,False,0.92,88.05,2023-01-13,2023-01,84.00,Mass,Malaga,ES,69,1733.48,2025-05-10,False,late_payer,0.5050,late_payer,60+,3,Subscriptions,True,8.95
1,2,1086,1700,Web,SETTLED,EUR,2,2023-05-12 13:45:22,22.60,0.000,False,1.00,22.60,2023-05-12,2023-05,22.60,Mass,Zaragoza,ES,82,2209.83,2025-07-08,True,travel,0.3745,travel,60+,1,Subscriptions,True,14.28
2,3,68944,2594,Web,SETTLED,EUR,1,2024-02-06 14:22:15,11.60,0.000,False,1.00,11.60,2024-02-06,2024-02,11.60,Mass,Bilbao,ES,49,798.13,2023-01-06,True,complaint,0.5761,complaint,46-60,4,Cards,False,21.02
3,4,27155,2179,App,DECLINED,EUR,1,2024-08-03 04:03:22,60.91,0.000,False,1.00,60.91,2024-08-03,2024-08,0.00,SME,Valencia,ES,78,2664.36,2023-02-23,True,late_payer,0.5943,late_payer,60+,5,Cards,False,21.53
4,5,70629,2643,Branch,SETTLED,EUR,1,2023-08-30 18:44:27,34.25,0.000,False,1.00,34.25,2023-08-30,2023-08,34.25,Mass,Zaragoza,ES,59,1033.51,2024-01-28,True,mortgage,0.6154,mortgage,46-60,5,Investments,True,17.27


In [36]:
txn = df_txn_enriched.copy()
txn["month"] = pd.to_datetime(txn["txn_ts"]).dt.to_period("M").dt.to_timestamp()

risk = df_risk_monthly.copy()
risk["month"] = pd.to_datetime(risk["month"])
risk = risk.drop_duplicates(subset=["customer_id", "month"], keep="last")

txn_with_risk_exact = txn.merge(
    risk,
    on=["customer_id", "month"],
    how="left",
    validate="many_to_one"
)

txn_with_risk_exact[["txn_id", "customer_id", "txn_ts", "month", "pd", "stage", "ead"]].head(10)


,txn_id,customer_id,txn_ts,month,pd,stage,ead
0,1,11700,2023-01-13 09:03:32,2023-01-01,NaN,<NA>,NaN
1,2,1086,2023-05-12 13:45:22,2023-05-01,NaN,<NA>,NaN
2,3,68944,2024-02-06 14:22:15,2024-02-01,NaN,<NA>,NaN
3,4,27155,2024-08-03 04:03:22,2024-08-01,NaN,<NA>,NaN
4,5,70629,2023-08-30 18:44:27,2023-08-01,0.07367,Stage2,2327.76
5,6,24230,2025-06-30 15:39:57,2025-06-01,NaN,<NA>,NaN
6,7,78928,2023-03-14 11:49:09,2023-03-01,NaN,<NA>,NaN
7,8,42063,2023-01-24 15:15:30,2023-01-01,NaN,<NA>,NaN
8,9,33475,2024-01-03 16:10:41,2024-01-01,0.02489,Stage2,1698.25
9,10,2028,2025-07-15 16:29:29,2025-07-01,0.05377,Stage2,2693.20


In [30]:
# top transacciones por importe
df_txn_fe.nlargest(10, "amount_eur")[["txn_id", "customer_id", "amount_eur", "currency", "txn_ts"]]


,txn_id,customer_id,amount_eur,currency,txn_ts
95989,95990,46132,3532.85,EUR,2023-09-30 20:34:13
318176,318177,59573,3232.16,EUR,2023-04-15 04:30:22
167500,167501,39229,2590.62,EUR,2025-09-03 15:45:09
345036,345037,14024,2487.29,EUR,2024-08-10 08:00:41
546248,546249,48275,2432.98,EUR,2023-05-12 15:13:03
550546,550547,36219,2304.93,GBP,2024-08-01 20:37:45
538372,538373,66830,2245.26,EUR,2025-04-09 13:35:29
480479,480480,20274,2094.44,EUR,2023-01-31 07:38:19
74037,74038,3867,2079.43,EUR,2025-08-31 04:02:55
140712,140713,37470,2079.38,EUR,2025-05-07 22:15:18


In [31]:
# ranking por cliente
df_rank = df_txn_fe[df_txn_fe["status"].eq("SETTLED")].copy()
df_rank["rank_within_customer"] = (
    df_rank.sort_values(["customer_id", "net_amount_eur"], ascending=[True, False])
           .groupby("customer_id")["net_amount_eur"]
           .rank(method="dense", ascending=False)
)
df_rank[["customer_id", "net_amount_eur", "rank_within_customer"]].head(15)


,customer_id,net_amount_eur,rank_within_customer
0,11700,84.00,1.0
1,1086,22.60,2.0
2,68944,11.60,5.0
4,70629,34.25,2.0
5,24230,-27.50,9.0
6,78928,66.09,4.0
7,42063,28.87,3.0
8,33475,11.04,2.0
9,2028,4.34,2.0
10,30256,21.68,6.0


In [32]:
# dedup ejemplo
df_tmp = pd.DataFrame({"a": [1,1,2,2,2], "b":[10,10,20,21,21]})
df_tmp.drop_duplicates()


,a,b
0,1,10
2,2,20
3,2,21


In [33]:
# sample reproducible
df_txn_fe.sample(5, random_state=42)


,txn_id,customer_id,product_id,channel,status,currency,quantity,txn_ts,amount,discount_pct,is_refund,fx_rate_to_eur,amount_eur,txn_date,txn_month,net_amount_eur
4242,4243,68267,1716,Branch,SETTLED,EUR,3,2023-08-06 00:01:54,60.14,0.000,False,1.0,60.14,2023-08-06,2023-08,60.14
60608,60609,14099,1473,App,SETTLED,EUR,2,2023-11-12 05:17:47,22.52,0.000,False,1.0,22.52,2023-11-12,2023-11,22.52
392832,392833,10979,2314,App,SETTLED,EUR,1,2023-10-03 15:44:35,3.80,0.000,False,1.0,3.80,2023-10-03,2023-10,3.80
41643,41644,71021,2744,App,SETTLED,EUR,1,2023-03-12 17:22:10,71.37,0.000,False,1.0,71.37,2023-03-12,2023-03,71.37
464234,464235,61069,2410,CallCenter,PENDING,EUR,1,2025-07-03 19:27:58,27.34,0.121,False,1.0,27.34,2025-07-03,2025-07,0.00


In [34]:
def add_month_and_flags(df):
    out = df.copy()
    out["month"] = out["txn_ts"].dt.to_period("M").astype(str)
    out["is_large"] = out["amount_eur"].abs() >= 1000
    return out

def compute_monthly_kpis(df):
    return (
        df[df["status"].eq("SETTLED")]
        .groupby(["month"], as_index=False)
        .agg(n_txn=("txn_id", "count"), net_sum=("net_amount_eur", "sum"), avg_net=("net_amount_eur", "mean"))
        .sort_values("month")
    )

monthly_kpis = (
    df_txn_fe
    .pipe(add_month_and_flags)
    .pipe(compute_monthly_kpis)
)

monthly_kpis.tail(12)


,month,n_txn,net_sum,avg_net
24,2025-01,14696,651740.58,44.348161
25,2025-02,13286,598621.68,45.056577
26,2025-03,14499,631098.71,43.527051
27,2025-04,13998,630601.33,45.049388
28,2025-05,14714,648814.45,44.095042
29,2025-06,14175,639821.42,45.137314
30,2025-07,14636,661555.15,45.200543
31,2025-08,14559,651139.38,44.724183
32,2025-09,14214,623979.51,43.898938
33,2025-10,14502,637897.30,43.986850


In [35]:
# Consejo: categoricals para strings repetitivos en fact tables
df_txn_opt = df_txn_fe.copy()
for col in ["status", "channel", "currency"]:
    df_txn_opt[col] = df_txn_opt[col].astype("category")

profile_df(df_txn_fe, "df_txn_fe (orig)")
profile_df(df_txn_opt, "df_txn_opt (categories)")



--- Profile: df_txn_fe (orig) ---
shape: (600000, 16)
dtypes:
 txn_id                     int64
customer_id                int64
product_id                 int64
channel                   object
status                    object
currency                  object
quantity                   int64
txn_ts            datetime64[ns]
amount                   float64
discount_pct             float64
is_refund                   bool
fx_rate_to_eur           float64
amount_eur               float64
txn_date                  object
txn_month                 object
net_amount_eur           float64
dtype: object
missing (%):
 channel           1.03
txn_id            0.00
customer_id       0.00
product_id        0.00
status            0.00
currency          0.00
quantity          0.00
txn_ts            0.00
amount            0.00
discount_pct      0.00
is_refund         0.00
fx_rate_to_eur    0.00
amount_eur        0.00
txn_date          0.00
txn_month         0.00
dtype: float64
memory (MB): 193.54


<div style="margin:18px 0 10px 0;padding:10px 12px;border-left:6px solid #009BC8;background:rgba(0,155,200,.08);border-radius:14px;border:1px solid rgba(0,0,0,.08);">
  <div style="font-size:18px;font-weight:850;">1. Exploración del problema</div>
  <div style="margin-top:2px;color:rgba(0,0,0,.65);font-size:13px;">
    Qué se responde aquí · supuestos · criterio de éxito
  </div>
</div>


### 1.1 Contexto y definición
2–4 líneas máximo. Luego, a trabajar.


<div style="padding:10px 12px;border-radius:14px;border:1px solid rgba(0,155,200,.35);background:rgba(0,155,200,.10);">
  <b style="color:#007FA6;">Info</b><br>
  <span style="color:rgba(0,0,0,.70);">
    Este notebook asume que el dataset ya está anonimizado. Si no lo está, detén aquí y aplica un pipeline de privacidad.
  </span>
</div>


<div style="padding:10px 12px;border-radius:14px;border:1px solid rgba(0,155,200,.25);background:rgba(0,155,200,.06);">
  <b style="color:#007FA6;">Tip</b><br>
  <span style="color:rgba(0,0,0,.70);">
    Mantén una semilla fija (<code>random_state</code>) y registra versiones de datos para reproducibilidad.
  </span>
</div>


<div style="padding:10px 12px;border-radius:14px;border:1px solid rgba(255,164,0,.35);background:rgba(255,164,0,.10);">
  <b style="color:rgba(0,0,0,.75);">Warning</b><br>
  <span style="color:rgba(0,0,0,.70);">
    Riesgo de leakage: revisa variables con timestamps posteriores al evento objetivo.
  </span>
</div>


<div style="padding:10px 12px;border-radius:14px;border:1px solid rgba(230,57,70,.25);background:rgba(230,57,70,.06);">
  <b style="color:rgba(0,0,0,.75);">Red flag</b><br>
  <span style="color:rgba(0,0,0,.70);">
    Si la métrica sube “demasiado” sin explicación, valida split, duplicados y features proxy del target.
  </span>
</div>


## Dataset Card

| Campo | Valor |
|---|---|
| Source | … |
| Owner | … |
| Refresh cadence | … |
| Rows / Columns | … / … |
| Time window | … |
| Target | `y` |
| Notas | Leakage risks, filtros, limitaciones… |
